# Basic imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import sys
import numpy as np
import pandas as pd
from pathlib import Path
print(sys.path)
sys.path.append("..")
sys.path.append("../..")
sys.path.append("../../..")

2022-10-10 23:11:30.215442: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-10 23:11:30.215462: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


['/home/hubert/librep-hiaac/experiments/Topological_ae', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages']


In [9]:
# from librep.transforms import TSNE
# from librep.transforms import UMAP
from librep.datasets.multimodal import TransformMultiModalDataset, ArrayMultiModalDataset, WindowedTransform
# from librep.metrics.dimred_evaluator import DimensionalityReductionQualityReport, MultiDimensionalityReductionQualityReport
# from librep.datasets.har.loaders import MNISTView
from sklearn.decomposition import PCA

In [31]:
from TopoAEMetrics import MeasureCalculator
from pyDRMetrics.pyDRMetrics import DRMetrics

# Loading MNIST

In [11]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
train_val_pd_X = np.reshape(x_train, (-1, 28*28))
train_val_pd_Y = y_train
test_pd_X = np.reshape(x_test, (-1, 28*28))
test_pd_Y = y_test

In [12]:
test_pd_X.shape

(10000, 784)

# Reduce with PCA

In [17]:
reducer = PCA(n_components=2)
train_val_pd_X_reduced = reducer.fit_transform(train_val_pd_X)
test_pd_X_reduced = reducer.fit_transform(test_pd_X)

In [19]:
train_val_pd_X_reduced.shape

(60000, 2)

In [20]:
test_pd_X_reduced.shape

(10000, 2)

In [21]:
test_pd_X_500 = test_pd_X[:500,:]
test_pd_X_500.shape

(500, 784)

In [22]:
test_pd_X_500_reduced = test_pd_X_reduced[:500,:]
test_pd_X_500_reduced.shape

(500, 2)

# Trustworthiness - TopoAE version

In [45]:
mcalculator = MeasureCalculator(test_pd_X_500, test_pd_X_500_reduced, 15)

In [46]:
mcalculator.trustworthiness(15)

0.7381003494060099

# Trustworthiness - pyDRMetrics version

In [32]:
drm = DRMetrics(test_pd_X_500, test_pd_X_500_reduced)

In [36]:
drm.T[10:20]

array([0.76225551, 0.7591624 , 0.75891416, 0.75847808, 0.75931505,
       0.75856579, 0.75962909, 0.75949265, 0.75954716, 0.75823882])

# Comparing execution time

In [50]:
import cProfile
import pstats
from pstats import SortKey

with cProfile.Profile() as pr:
    drm = DRMetrics(test_pd_X_500, test_pd_X_500_reduced)
    print(drm.T[10:20])
    mcalculator = MeasureCalculator(test_pd_X_500, test_pd_X_500_reduced, 15)
    print(mcalculator.trustworthiness(15))
    stats = pstats.Stats(pr)
    stats.sort_stats(SortKey.CUMULATIVE).print_stats(10)
    stats.sort_stats(SortKey.TIME).print_stats(10)

[0.76225551 0.7591624  0.75891416 0.75847808 0.75931505 0.75856579
 0.75962909 0.75949265 0.75954716 0.75823882]
0.7381003494060099
         54555 function calls (51997 primitive calls) in 0.413 seconds

   Ordered by: cumulative time
   List reduced from 339 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.335    0.335 /usr/local/lib/python3.8/dist-packages/pyDRMetrics/pyDRMetrics.py:22(__init__)
6549/4018    0.005    0.000    0.157    0.000 {built-in method numpy.core._multiarray_umath.implement_array_function}
        1    0.145    0.145    0.145    0.145 /usr/local/lib/python3.8/dist-packages/pyDRMetrics/coranking_matrix.py:40(coranking_matrix)
     2506    0.106    0.000    0.106    0.000 {method 'argsort' of 'numpy.ndarray' objects}
     2004    0.001    0.000    0.093    0.000 <__array_function__ internals>:177(argsort)
     2004    0.001    0.000    0.092    0.000 /usr/local/lib/python3.8/dis